<a href="https://colab.research.google.com/github/Mahjabeenqamar1/Sentiment-Analysis-LSTM-/blob/main/Sentiment_Analysis_(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
from nltk.corpus import stopwords
import re
import joblib
from keras.preprocessing.text import Tokenizer
import gensim
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Data_set/training.1600000.processed.noemoticon.csv",encoding='latin-1',header=None)

In [ ]:
df.head()



,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In the Dataset we can see that there is no column names present so we add the column names of the Data.


In [ ]:


columns=['target','ids','date','flag','user','text']
df.columns=columns



In [ ]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Data Preprocessing

Tweets contains a lot of emoticons, abbreviations and creative ways of expressing excitment such as long tailing (ex. happyyyy). We normalize all letters to lowercase and remove any ”@USERNAME” and ”#hashtag” because they did not affect the sentiment of text.

Removing Stopwords

Stemming

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Replacing the target Values by Positive,Negative and Neutral

In [ ]:
df.head()

,target,ids,date,flag,user,text
0,Negative,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,Negative,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,Negative,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,Negative,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,Negative,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
stop_words=set(stopwords.words('english'))
stop_words.remove('not')

In [ ]:
corpus=[]
for i in range(0,len(df)):
    review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',df['text'][i])
    review=review.lower()
    review=review.split()
    review=[word for word in review if not word in stop_words]
    review=' '.join(review)
    corpus.append(review)




Replacing the text column with preprocessed text


In [ ]:
df.text=corpus

In [ ]:
df.head()

,target,ids,date,flag,user,text
0,Negative,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,awww bummer shoulda got david carr third day
1,Negative,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset update facebook texting might cry result...
2,Negative,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,dived many times ball managed save 50 rest go ...
3,Negative,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feels itchy like fire
4,Negative,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,not behaving mad see




Splitting the Data into Training and Test set


In [ ]:
from sklearn.model_selection import train_test_split
train_df,test_df=train_test_split(df,test_size=0.20,random_state=123)


In [ ]:
train_df.head()

,target,ids,date,flag,user,text
1578903,Positive,2189875205,Tue Jun 16 01:08:25 PDT 2009,NO_QUERY,silent_serenade,withholding information shae yup
51568,Negative,1679002293,Sat May 02 08:17:21 PDT 2009,NO_QUERY,tommytrc,uhhh ummmmm yes dear keep coming tweetups prom...
569210,Negative,2207868145,Wed Jun 17 08:29:12 PDT 2009,NO_QUERY,nikkikikicoco,amazing time wannqa come home come back suitca...
71896,Negative,1694083746,Mon May 04 01:00:43 PDT 2009,NO_QUERY,bszafko,ouch wrist hurts gym accident see surgeon
405089,Negative,2058584282,Sat Jun 06 15:13:45 PDT 2009,NO_QUERY,carolduff,going crazy work school



Word2Vec Model


In [ ]:
documents = [text.split() for text in train_df.text]

In [ ]:
w2v_model = gensim.models.word2vec.Word2Vec(size=300, ## one hot encodeind
                                            window=7,          
                                            min_count=10, 
                                            workers=8)



In [ ]:
w2v_model.build_vocab(documents)


In [ ]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 30446


In [ ]:


w2v_model.train(documents, total_examples=len(documents), epochs=30)



(248021754, 279410310)

In [ ]:


w2v_model.wv.most_similar("good")



[('great', 0.6885119676589966),
 ('goood', 0.5697886943817139),
 ('bad', 0.5605125427246094),
 ('gooood', 0.5314722061157227),
 ('gooooood', 0.5167181491851807),
 ('nice', 0.48782533407211304),
 ('gd', 0.4663253724575043),
 ('gud', 0.4651547074317932),
 ('rough', 0.45763182640075684),
 ('goooood', 0.4545465111732483)]

In [ ]:
w2v_model.wv.most_similar("hate")

[('hates', 0.551250696182251),
 ('sucks', 0.4870941936969757),
 ('suck', 0.4769120216369629),
 ('stupid', 0.47291573882102966),
 ('h8', 0.43198832869529724),
 ('dislike', 0.4319857954978943),
 ('annoying', 0.42702531814575195),
 ('hating', 0.42481520771980286),
 ('despise', 0.4221572279930115),
 ('ugh', 0.4125872850418091)]

In [ ]:
w2v_model.wv.most_similar("great")

[('fantastic', 0.73514723777771),
 ('wonderful', 0.6997291445732117),
 ('awesome', 0.692935585975647),
 ('good', 0.6885119676589966),
 ('fabulous', 0.6531974077224731),
 ('amazing', 0.6460429430007935),
 ('fab', 0.6041921377182007),
 ('excellent', 0.583431601524353),
 ('nice', 0.5736099481582642),
 ('gr8', 0.5569304823875427)]

In [ ]:


tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df.text)



In [ ]:


tokenizer.word_index



{'not': 1,
 'good': 2,
 'day': 3,
 'get': 4,
 'like': 5,
 'go': 6,
 'quot': 7,
 'today': 8,
 'work': 9,
 'love': 10,
 'going': 11,
 'got': 12,
 'lol': 13,
 'time': 14,
 'back': 15,
 'u': 16,
 'one': 17,
 'know': 18,
 'im': 19,
 'really': 20,
 'amp': 21,
 'see': 22,
 'night': 23,
 'still': 24,
 '2': 25,
 'well': 26,
 'want': 27,
 'new': 28,
 'think': 29,
 'home': 30,
 'thanks': 31,
 'oh': 32,
 'much': 33,
 'miss': 34,
 'need': 35,
 'last': 36,
 'morning': 37,
 'tomorrow': 38,
 'hope': 39,
 'great': 40,
 'twitter': 41,
 '3': 42,
 'haha': 43,
 'feel': 44,
 'sad': 45,
 'fun': 46,
 'wish': 47,
 'sleep': 48,
 'right': 49,
 'would': 50,
 'bad': 51,
 'happy': 52,
 'sorry': 53,
 'tonight': 54,
 'come': 55,
 'make': 56,
 'way': 57,
 'getting': 58,
 'though': 59,
 'gonna': 60,
 'nice': 61,
 'better': 62,
 'watching': 63,
 'yeah': 64,
 'wait': 65,
 'bed': 66,
 'could': 67,
 'week': 68,
 'school': 69,
 'people': 70,
 'hate': 71,
 'days': 72,
 'even': 73,
 'hey': 74,
 'next': 75,
 '4': 76,
 'yes': 7

In [ ]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

290653

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df.text), maxlen=300)
X_train

array([[    0,     0,     0, ...,  3008, 21821,   796],
       [    0,     0,     0, ...,     7,   531,  2976],
       [    0,     0,     0, ...,  9856,   874,   109],
       ...,
       [    0,     0,     0, ...,   262,   774,     1],
       [    0,     0,     0, ...,   158, 17570,    11],
       [    0,     0,     0, ...,     7, 32054,     7]], dtype=int32)

In [ ]:
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df.text), maxlen=300)
X_test

array([[    0,     0,     0, ...,   256,  1664,   199],
       [    0,     0,     0, ...,  6190,  1790,    75],
       [    0,     0,     0, ...,  4779,   806,   111],
       ...,
       [    0,     0,     0, ...,   572, 66636,  1913],
       [    0,     0,     0, ...,   471,    82,   198],
       [    0,     0,     0, ...,   203,   337,  5137]], dtype=int32)

In [ ]:
y_train=train_df.target
y_train.head()

1578903    Positive
51568      Negative
569210     Negative
71896      Negative
405089     Negative
Name: target, dtype: object

In [ ]:
y_test=test_df.target
y_test.head()

448282     Negative
1475261    Positive
132529     Negative
182348     Negative
907614     Positive
Name: target, dtype: object



Encoding the Categorical target into 0 and 1


In [ ]:
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_test=labelencoder.fit_transform(y_test)



In [ ]:
y_train.shape


(1280000,)

In [ ]:
y_test.shape

(320000,)



Embedding Matrix


In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(290653, 300)


In [ ]:
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)



Build Model using LSTM


In [ ]:


model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          87195900  
_________________________________________________________________
dropout (Dropout)            (None, 300, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 87,356,401
Trainable params: 160,501
Non-trainable params: 87,195,900
_________________________________________________________________


Compile Model

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
model_history=model.fit(X_train, y_train,batch_size=1024,epochs=5,validation_split=0.1,verbose=1)

Epoch 1/5
1125/1125 [==============================] - 13770s 12s/step - loss: 0.4993 - accuracy: 0.7543 - val_loss: 0.4576 - val_accuracy: 0.7855
Epoch 2/5
  54/1125 [>.............................] - ETA: 3:26:26 - loss: 0.4810 - accuracy: 0.7663



Fitting the Model


In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs=range(len(acc))


In [ ]:


plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")



In [ ]:
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")



Preprocessing of tweets given by user


In [ ]:


def preprocess(text):
    review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',text)
    review=review.lower()
    review=review.split()
    review=[word for word in review if not word in stop_words]
    print(review)
    review=pad_sequences(tokenizer.texts_to_sequences([review]), maxlen=300)
    return review





Prediction


In [ ]:
def prediction(review):
    review=preprocess(review)
    score=model.predict(review)
    score=score[0]
    if score<0.4:
        print("Negative")
    elif score>0.4 and score<0.6:
        print("Neutral")
    else:
        print("Positive")
    print(score)

In [ ]:
prediction("the food is not bad")


In [ ]:


prediction("the film was horrible")



In [ ]:


scores = model.predict(X_test, verbose=1, batch_size=1024)



In [ ]:
scores


In [ ]:


y_pred=np.where(scores>0.5,1,0)



In [ ]:


y_pred



In [ ]:
y_test




Evaluation Using Confusion Matrix, accuracy_score and classification report


In [ ]:
cm=confusion_matrix(y_pred,y_test)
print(cm)

In [ ]:
print(accuracy_score(y_pred,y_test))

In [ ]:
print(classification_report(y_test, y_pred))



Saving the Trained Models


In [ ]:


joblib.dump(w2v_model,'word2vec.pkl')
joblib.dump(tokenizer,'tokenizer.pkl')
joblib.dump(model,'final_model.pkl')



https://www.kaggle.com/prashant268/sentiment-analysis-lstm/notebook